# Scraping article content

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import time
import openai
import redis
import json
import sqlite3
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class ArmenpressCrawler(scrapy.Spider):
    name = "MovsisyansNewsCrawler"

    custom_settings=dict(
        DOWNLOAD_DELAY = 0.08
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.domain = r"https://armenpress.am/eng/news/"
        self.r = redis.Redis("localhost", 5211, decode_responses=True)

    def start_requests(self):
        while True:
            to_fetch = self.r.rpop("to_fetch")
            if to_fetch is None:
                time.sleep(2)
            else:
                yield scrapy.Request(url=self.domain + str(to_fetch), callback=self.parse_article)

    def parse_article(self, response):
        print(f"SCRAPING {response.url}")
        resp_text = " ".join(response.text.split())
        (title := " ".join(re.findall('<h1 class="opennewstitle" itemprop="name">(.+)</h1>', resp_text)[0].split()))
        print((content := " ".join(response.css('span[itemprop="articleBody"]::text').extract_first().split())))
        id = int(re.findall(r"\d+", response.url)[0])

        self.r.lpush("to_store", json.dumps({"id": id, "title": title, "content": content}))
        

In [3]:
proc = CrawlerProcess()
proc.crawl(ArmenpressCrawler)
proc.start()

2023-11-19 23:36:51 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-11-19 23:36:51 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Windows-10-10.0.22621-SP0
2023-11-19 23:36:51 [scrapy.addons] INFO: Enabled addons:
[]
2023-11-19 23:36:51 [py.warnings] WARNING: c:\Projects\NewsEntityTracking\.venv\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documen

2023-11-19 23:36:51 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2023-11-19 23:36:51 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 0.08}
2023-11-19 23:36:51 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddle

In [ ]:
# response = requests.get("https://armenpress.am/eng/news/218020")
# resp_text = " ".join(response.text.split())
# (title := re.findall('<h1 class="opennewstitle" itemprop="name">(.+)</h1>', resp_text)[0])
# (content := re.findall('itemprop="articleBody"><?p?>?(.+)<?/?p?>?</span>', resp_text)[0])

# id = int(re.findall(r"\d+", response.url)[0])

In [ ]:
response.css('span[itemprop="articleBody"].last-child')